<a href="https://colab.research.google.com/github/NandakrishnanR/Kaggle_projects/blob/master/DecisionTree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("jsphyg/weather-dataset-rattle-package")

print("Path to dataset files:", path)


ModuleNotFoundError: No module named 'kagglehub'

In [ ]:
import pandas as pd
raw_df=pd.read_csv(f"/kaggle/input/weather-dataset-rattle-package/weatherAUS.csv")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import matplotlib
import os
%matplotlib inline

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 150)
sns.set_style('darkgrid')
matplotlib.rcParams['font.size'] = 14
matplotlib.rcParams['figure.figsize'] = (10, 6)
matplotlib.rcParams['figure.facecolor'] = '#00000000'

In [ ]:
print(raw_df.isnull().sum())

Date                 0
Location             0
MinTemp           1485
MaxTemp           1261
Rainfall          3261
Evaporation      62790
Sunshine         69835
WindGustDir      10326
WindGustSpeed    10263
WindDir9am       10566
WindDir3pm        4228
WindSpeed9am      1767
WindSpeed3pm      3062
Humidity9am       2654
Humidity3pm       4507
Pressure9am      15065
Pressure3pm      15028
Cloud9am         55888
Cloud3pm         59358
Temp9am           1767
Temp3pm           3609
RainToday         3261
RainTomorrow      3267
dtype: int64


In [ ]:
#Given that 'RainTomorrow' is almost certainly your target variable for a prediction task, it's generally best to remove the rows where 'RainTomorrow' is null. Imputing a categorical target variable can lead to your model learning from artificially generated labels, making your model's evaluation and predictions unreliable. You have 3267 missing values for 'RainTomorrow' out of 145,460 entries, which is about 2.2%. Removing these rows is a reasonable approach to ensure data quality for your target variable.
raw_df.dropna(subset = ['RainTomorrow'], inplace = True)


## Preparing the Data for Training

steps to prepare the dataset for training:

1. Create a train/test/validation split
2. Identify input and target columns
3. Identify numeric and categorical columns
4. Impute (fill) missing numeric values
5. Scale numeric values to the $(0, 1)$ range
6. Encode categorical columns to one-hot vectors

In [ ]:
year = pd.to_datetime(raw_df['Date']).dt.year
raw_df['year'] = year
from plotly import express as px
px.histogram(raw_df, x='year', color='RainToday', title='Rain Today by Year')

In [ ]:
train_df=raw_df[raw_df['year']<2016]
val_df=raw_df[raw_df['year'] == 2016]
test_df=raw_df[raw_df['year'] > 2016]

In [ ]:
input_col=list(raw_df.columns)
input_col.remove('RainTomorrow')
input_col.remove('Date')
input_col.remove('year')
target_col='RainTomorrow'
train_input=train_df[input_col].copy()
train_target=train_df[target_col].copy()
val_input=val_df[input_col].copy()
val_target=val_df[target_col].copy()
test_input=test_df[input_col].copy()
test_target=test_df[target_col].copy()


,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday
0,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,WNW,20.0,24.0,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No
1,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,WSW,4.0,22.0,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No
2,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,WSW,19.0,26.0,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No
3,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,E,11.0,9.0,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No
4,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,NW,7.0,20.0,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144913,Uluru,20.5,34.7,0.0,NaN,NaN,E,52.0,ESE,E,35.0,20.0,23.0,12.0,1013.2,1010.1,NaN,NaN,24.3,33.0,No
144914,Uluru,18.0,36.4,0.0,NaN,NaN,ESE,54.0,E,ESE,30.0,31.0,17.0,7.0,1014.7,1010.9,NaN,NaN,26.7,35.0,No
144915,Uluru,17.5,37.1,0.0,NaN,NaN,E,56.0,E,SE,33.0,22.0,12.0,7.0,1012.6,1007.5,NaN,NaN,28.1,34.7,No
144916,Uluru,20.0,38.9,0.0,NaN,NaN,E,59.0,E,SSE,20.0,17.0,12.0,12.0,1007.2,1002.6,NaN,1.0,31.3,38.4,No


In [ ]:
## numerical columns
# only training set needs fitting; others are transformed using the same imputer
numerical_cols = list(train_input.select_dtypes(include=['int64', 'float64']))
categorical_cols = list(train_input.select_dtypes(include=['object']))
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='mean')
imputer.fit(train_input[numerical_cols])

train_input[numerical_cols] = imputer.transform(train_input[numerical_cols])
val_input[numerical_cols]   = imputer.transform(val_input[numerical_cols])
test_input[numerical_cols]  = imputer.transform(test_input[numerical_cols])


## categorical cols — one-hot encoding creates many columns; use a safe concat-based approach
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

# Robust selection: include pandas string-like dtypes too
cat_cols = train_input.select_dtypes(include=['object','string','category']).columns.tolist()

if len(cat_cols) == 0:
    print("No categorical columns found in train_input. Skipping one-hot encoding.")
    enc_cols = []
else:
    # Fit on TRAIN ONLY (no leakage)
    ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
    ohe.fit(train_input[cat_cols])

    # Get encoded feature names; no-arg avoids mismatch
    enc_cols = ohe.get_feature_names_out().tolist()

    # Transform each split and wrap in DataFrames to keep names and index alignment
    train_cat = pd.DataFrame(ohe.transform(train_input[cat_cols]), columns=enc_cols, index=train_input.index)
    val_cat   = pd.DataFrame(ohe.transform(val_input[cat_cols]),   columns=enc_cols, index=val_input.index)
    test_cat  = pd.DataFrame(ohe.transform(test_input[cat_cols]),  columns=enc_cols, index=test_input.index)

    # Replace original categorical columns in a single concat (avoids fragmentation)
    train_input = pd.concat([train_input.drop(columns=cat_cols), train_cat], axis=1)
    val_input   = pd.concat([val_input.drop(columns=cat_cols),   val_cat], axis=1)
    test_input  = pd.concat([test_input.drop(columns=cat_cols),  test_cat], axis=1)

    # Optional: force block consolidation if prior ops fragmented the frame
    train_input = train_input.copy()
    val_input   = val_input.copy()
    test_input  = test_input.copy()

print(f"Numeric columns: {len(numerical_cols)}; Encoded categorical columns: {len(enc_cols)}")
print("train_input shape:", train_input.shape)

AssertionError: No categorical columns found in train_input.